In [1]:
import pandas as pd
import plotly.express as px
from datetime import datetime
from dateutil import parser
import numpy as np
px.set_mapbox_access_token("pk.eyJ1Ijoidm9sY2FubzE5OTgiLCJhIjoiY2s4bDI3OGhiMDg2ejNzbnc3M2F6ZG4waiJ9.QsBbnFD07hqZP0iRXsrjNA")


/opt/anaconda3/lib/python3.7/site-packages/IPython/utils/traitlets.py:5: UserWarning:

IPython.utils.traitlets has moved to a top-level traitlets package.



# Overview

### I compared data of Feb and May. Mostly I compare the ridership through boarding number.

In [3]:
df_Feb=pd.read_csv('data/APC_1_6-16-2020/APC_Feb.csv')

In [4]:
df_May=pd.read_csv('data/APC_1_6-16-2020/APC_May.csv')

In [5]:
bd_feb=df_Feb.groupby(['Stop Id','Rid Check Date'])[['Board']].sum().groupby(['Stop Id'])[['Board']].mean().reset_index()
stop_feb=df_Feb.groupby('Stop Id')['Lon','Lat'].mean().reset_index()
stop_inf_feb=pd.merge(bd_feb,stop_feb)
stop_inf_feb[['Lon','Lat']]=stop_inf_feb[['Lon','Lat']].apply(lambda x: x/1000000)

## Board number per day in Feb

In [6]:
fig_Feb = px.scatter_mapbox(stop_inf_feb, lat="Lat", lon="Lon",     color="Board", size="Board",
                  color_continuous_scale=px.colors.cyclical.IceFire, size_max=15, zoom=10)
fig_Feb.show()

In [7]:
bd_may=df_May.groupby(['Stop Id','Rid Check Date'])[['Board']].sum().groupby(['Stop Id'])[['Board']].mean().reset_index()
stop_may=df_May.groupby('Stop Id')['Lon','Lat'].mean().reset_index()
stop_inf_may=pd.merge(bd_may,stop_may)
stop_inf_may[['Lon','Lat']]=stop_inf_may[['Lon','Lat']].apply(lambda x: x/1000000)

## Board number per day in May

In [8]:
fig_May = px.scatter_mapbox(stop_inf_may, lat="Lat", lon="Lon", color="Board", size="Board",
                  color_continuous_scale=px.colors.cyclical.IceFire, size_max=15, zoom=10)
fig_May.show()

In [9]:
df_Feb['wkday']=df_Feb['Rid Check Date'].apply(lambda x:datetime. strptime(x, '%m/%d/%y').weekday())

In [10]:
df_May['wkday']=df_May['Rid Check Date'].apply(lambda x:datetime. strptime(x, '%m/%d/%y').weekday())

In [11]:
wk_f=df_Feb.groupby(['wkday','Rid Check Date'])[['Board']].sum().groupby('wkday')[['Board']].mean().reset_index()
wk_m=df_May.groupby(['wkday','Rid Check Date'])[['Board']].sum().groupby('wkday')[['Board']].mean().reset_index()

In [12]:
wk_f['Month']='Feb'
wk_m['Month']='May'
wk_inf=pd.concat([wk_f,wk_m],ignore_index=True)
wk_inf

,wkday,Board,Month
0,0,15222.250000,Feb
1,1,15312.500000,Feb
2,2,16053.000000,Feb
3,3,16218.750000,Feb
4,4,15256.750000,Feb
5,5,7983.250000,Feb
6,6,6439.333333,Feb
7,0,5784.250000,May
8,1,6498.500000,May
9,2,6236.250000,May


## Board number change over a week (Feb vs May)

In [13]:
df = px.data.tips()
fig = px.bar(wk_inf, x="wkday", y="Board", color='Month', barmode='group',
             height=400)
fig.show()

## Board number distribution in Feb (one observation: one stop)

In [14]:
fig = px.histogram(stop_inf_feb, x="Board", nbins=50)
fig.show()

#### Most part are in 0~120 in Feb

## Board number distribution in May (one observation: one stop)

In [15]:

fig = px.histogram(stop_inf_may, x="Board", nbins=50)
fig.show()


#### Most part are in 0~50 in May

## Quantile Comparison

In [133]:
np.quantile(stop_inf_feb.Board,0.95)

30.57596153846154

In [134]:
np.quantile(stop_inf_may.Board,0.95)

15.540258064516113

5% of stops have more than 30 people on boarding per day in Feb,

5% of stops have more than 15 people on boarding per day in May

## the boarding number difference

In [16]:
stop_inf_bd=pd.merge(stop_inf_feb,stop_inf_may,left_on='Stop Id',right_on='Stop Id')
stop_inf_bd['bd_diff']=stop_inf_bd['Board_y']-stop_inf_bd['Board_x']
stop_inf_bd['bd_diff_pct']=stop_inf_bd['bd_diff']/stop_inf_bd['Board_x']

In [18]:
top_decrease=stop_inf_bd.sort_values(['bd_diff'])[:100]

### below is the top 100 stops which have the greatest reduction of boarding number from Freb to May

In [21]:
top_decrease

,Stop Id,Board_x,Lon_x,Lat_x,Board_y,Lon_y,Lat_y,bd_diff,bd_diff_pct
1081,4420,632.481481,-86.781468,36.166358,167.193548,-86.781468,36.166358,-465.287933,-0.735655
1086,4428,495.851852,-86.781188,36.166456,175.935484,-86.781188,36.166456,-319.916368,-0.645185
1090,4434,472.777778,-86.781973,36.166554,189.774194,-86.781973,36.166554,-283.003584,-0.598597
1074,4413,463.666667,-86.781917,36.166581,191.709677,-86.781917,36.166581,-271.956989,-0.586536
1089,4432,366.576923,-86.781693,36.166706,135.566667,-86.781693,36.166706,-231.010256,-0.630182
1083,4423,479.814815,-86.781682,36.166215,261.161290,-86.781682,36.166215,-218.653524,-0.455704
1080,4419,270.592593,-86.781827,36.166233,98.107143,-86.781827,36.166233,-172.485450,-0.637436
1077,4416,214.259259,-86.781412,36.166751,90.700000,-86.781412,36.166751,-123.559259,-0.576681
1084,4424,220.518519,-86.781480,36.166358,102.125000,-86.781480,36.166358,-118.393519,-0.536887
1088,4431,189.000000,-86.781996,36.166590,78.096774,-86.781996,36.166590,-110.903226,-0.586790


### Show the top 10 stops in map

In [19]:
fig_May = px.scatter_mapbox(top_decrease, lat="Lat_x", lon="Lon_x", color=-top_decrease["bd_diff"], size=-top_decrease["bd_diff"],
                  color_continuous_scale=px.colors.cyclical.IceFire, size_max=15, zoom=10)
fig_May.show()